# Querying
Record times to execute queries in different databases of variable sizes

### Libraries

In [2]:
import pandas as pd
import mysql.connector
import os
import time
import pymongo
import datetime

In [3]:
from insert import ins

In [4]:
%load_ext dotenv
%dotenv

### Some Constants

In [5]:
MYSQL = "MySQL"
MONGO_DB = 'MongoDB'

In [6]:
THOUSAND = 'thousand'
LAKH = 'lakh'
FIVE_MILLION = 'five_mil'

### Query

In [7]:
def sql_query(query, cursor):
    begin = time.time()
    cursor.execute(query)
    cursor.fetchall()
    end = time.time()
    return end - begin

def mongo_query(db, col, query):
    begin = time.time()
    db[col].aggregate(query)
    end = time.time()
    return end - begin

In [49]:
def exp(size):
    ins(size)
    obs = {}
    # MySQL
    print()
    obs[MYSQL] = {}
    try:
        print("Connecting to MySQL")
        conn = mysql.connector.connect(
            host='localhost',
            database='social',
            user=os.getenv('MYSQL_USER'),
            password=os.getenv('MYSQL_PASS'),
            auth_plugin='mysql_native_password'
        )
        cursor = conn.cursor()
        print("Executing MySQL queries")
        # (1)
        query = """
            SELECT name, surname 
            FROM user JOIN comment USING (user_id);
        """
        obs[MYSQL][1] = sql_query(query, cursor)
        # (2)
        query = """
            SELECT * FROM user
            JOIN comment USING (user_id)
            WHERE text LIKE "%today%";
        """
        obs[MYSQL][2] = sql_query(query, cursor)
        # (3)
        query = """
            SELECT
                name,
                surname,
                created_at
            FROM user JOIN comment USING (user_id)
            WHERE
                created_at >= CURRENT_DATE AND
                created_at < DATE_ADD(CURRENT_DATE, INTERVAL 1 DAY);
        """
        obs[MYSQL][3] = sql_query(query, cursor)
        conn.close()
    except mysql.connector.Error as error:
        print(f"Failed to connect to MySQL: {error}")
    # MongoDB
    print()
    obs[MONGO_DB] = {}
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['test']
    print("Executing MongoDB queries")
    # (1)
    query = [
        {
            '$lookup': {
                'from': 'comment',
                'localField': '_id',
                'foreignField': 'user_id',
                'as': 'comments',
            },
        },
        {
            '$match': {
                'comments': { '$not': { '$size': 0 } }
            }
        }
    ]
    obs[MONGO_DB][1] = mongo_query(db, 'user', query)
    # (2)
    query = [
        {
            '$match': {
                'text': { '$regex': 'today' }
            },
        },
        {
            '$lookup': {
                'from': 'user',
                'localField': 'user_id',
                'foreignField': '_id',
                'as': 'user',
            }
        }
    ]
    obs[MONGO_DB][2] = mongo_query(db, 'comment', query)
    # (3)
    query = [
        {
            '$match': {
                'created_at': { '$regex': f'^{datetime.date.today()}' }
            },
        },
        {
            '$lookup': {
                'from': 'user',
                'localField': 'user_id',
                'foreignField': '_id',
                'as': 'author',
            }
        }
    ]
    obs[MONGO_DB][3] = mongo_query(db, 'comment', query)
    # Redis
    print()
    print(obs)

## Thousand

In [50]:
exp(THOUSAND)

Loading thousand

MySQL
----------
Deleting old records
Inserting users
##########
Inserting comments
##########

MongoDB
----------
Deleting old records
Inserting users
####################
Inserting comments
####################

Connecting to MySQL
Executing MySQL queries

Executing MongoDB queries

{'MySQL': {1: 0.008970975875854492, 2: 0.001995563507080078, 3: 0.00099945068359375}, 'MongoDB': {1: 0.10402965545654297, 2: 0.0009958744049072266, 3: 0.0020084381103515625}}


## Lakh

In [51]:
exp(LAKH)

Loading lakh

MySQL
----------
Deleting old records
Inserting users
##########
Inserting comments
##########

MongoDB
----------
Deleting old records
Inserting users
####################
Inserting comments
####################

Connecting to MySQL
Executing MySQL queries

Executing MongoDB queries

{'MySQL': {1: 0.6710329055786133, 2: 0.2006669044494629, 3: 0.09599971771240234}, 'MongoDB': {1: 6.574007511138916, 2: 0.061994075775146484, 3: 0.05903172492980957}}


## Five Million

In [ ]:
exp(FIVE_MILLION)

Loading five_mil
Connecting to MySQL
Executing MySQL queries
{'MySQL': {1: 34.89011740684509, 2: 8.41347074508667, 3: 5.430591106414795}}
